In [1]:
# !pip3 install minio
# !pip3 install pandas
# !sudo apt-get install -y mosquitto-clients

In [2]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

#df_metrics = pd.DataFrame(columns=['Workload_size','Workload TPT','Average Workload TPT','Average FET','Average CT'])
http://172.17.141.197:3000/goto/MU1mA-MVk?orgId=1

In [29]:
#scenario="emqx_testing_namespace_scaling_without_max_inflight_replicas_2_10_3"
scenario = "syncrnous_sdp_30s"
user=1
iteration=1
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'


In [4]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv


In [5]:
#!cp /home/ubuntu/scaling-sdp/aeneas/dataset/p001.mp3 .

In [6]:
!locust --headless -f mqtt_locust.py --host='127.0.0.1' -t 10  --users 2 --spawn-rate 2

[2022-09-23 12:23:19,502] ubuntu/WARNING/locust.main: System open file limit '4096' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2022-09-23 12:23:19,504] ubuntu/INFO/locust.main: Run time limit set to 10 seconds
[2022-09-23 12:23:19,504] ubuntu/INFO/locust.main: Starting Locust 2.12.0
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2022-09-23 12:23:19,507] ubuntu/INFO/locust.runners: Ramping to 2 users at a rate of 2.00 per second
[2022-09-23 12:23:19,509] ubuntu/INFO/locust.runners:

In [7]:
#cmd = "./mqtt_client.sh "+str(user)
#with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
#    output1 = process.communicate()[0].decode("utf-8")
time.sleep(100)

In [8]:

df= pd.read_csv('input_data.csv')
print(df)
df_metrics=pd.read_csv('sdp_metrics.csv')

                         intime
0  '2022-09-23 12:23:19.534462'
1  '2022-09-23 12:23:19.541247'


In [9]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)


In [10]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime']).dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'])
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2022-09-23 12:23:19.534462,2022-09-23 12:23:26.028
1,2022-09-23 12:23:19.541247,2022-09-23 12:23:28.379


0 days 00:00:08.844538


In [11]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))    


In [17]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-xlhm8  -n openfaas -c gateway  | grep /function/aeneas | cut -c 81-85 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-xlhm8 -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
df

,intime,outtime,aeneas,tocloud,TPT
0,2022-09-23 12:23:19.534462,2022-09-23 12:23:26.028,5.083,0.0404,6
1,2022-09-23 12:23:19.541247,2022-09-23 12:23:28.379,6.605,0.0420,8


In [18]:
#print(df['aeneas'])
#print(df['tocloud'])
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime']).dt.seconds
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
df['Waiting time']= (df['TPT']).astype(float) - df['FET']

display(df)
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario


,intime,outtime,aeneas,tocloud,TPT,FET,Waiting time
0,2022-09-23 12:23:19.534462,2022-09-23 12:23:26.028,5.083,0.0404,6,5.1234,0.8766
1,2022-09-23 12:23:19.541247,2022-09-23 12:23:28.379,6.605,0.0420,8,6.6470,1.3530


In [19]:
df_metrics = df_metrics.append(df, ignore_index=True)
df_metrics.to_csv("sdp_metrics.csv",index=False)

/tmp/ipykernel_197396/2485122144.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(df, ignore_index=True)


In [20]:
user=df.shape[0]
metrics = {'workload_type':str(user)+"_"+scenario,'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean(),'Average Waiting Time':df['Waiting time'].mean()}


In [21]:
pre_url = PROMETHEUS + '/api/v1/query?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time = (time.mktime(timestamp.timetuple()))
#interval= int(df['TPT'].sum()) if int(df['TPT'].sum()) > 60  else 60


In [22]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [23]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

60


In [24]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    data=res.get('data').get('result')[0].get('value')[1]
    return data

In [30]:
expr_cpu='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace=~"openfaas.*|default"}[5m]))&time='+str(time)
expr_memory='sum(container_memory_working_set_bytes{container_name!="POD",namespace=~"openfaas.*|default"})&time='+str(time)
expr_disk_read ='sum(rate(container_fs_reads_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
expr_disk_write='sum(rate(container_fs_writes_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
expr_network_transmit='sum(rate(container_network_transmit_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
expr_network_recieve='sum(rate(container_network_receive_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
#expr_cpu = '100 - (avg (irate(node_cpu_seconds_total{mode="idle"}['+str(interval)+'s])) * 100)&time='+str(time)
#expr_memory = '100 - ((sum(node_memory_MemAvailable_bytes) / sum(node_memory_MemTotal_bytes))*100)&time='+str(time)
#expr_disk_read = 'sum(rate(node_disk_read_bytes_total['+str(interval)+'s]))&time='+str(time)
#expr_disk_write =  'sum(rate(node_disk_written_bytes_total['+str(interval)+'s]))&time='+str(time)
#expr_network_transmit= 'sum(rate(node_network_transmit_errs_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
#expr_network_recieve='sum(rate(node_network_receive_packets_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
metrics['C_CPU'] = getdataprometheus(pre_url+expr_cpu)
metrics['C_RAM'] = getdataprometheus(pre_url+expr_memory)
metrics['C_disk_read']=getdataprometheus(pre_url+expr_disk_read)
metrics['C_disk_write']=getdataprometheus(pre_url+expr_disk_write)
metrics['C_network_recieve']=getdataprometheus(pre_url+expr_network_transmit)
metrics['C_network_transmit']=getdataprometheus(pre_url+expr_network_recieve)

ConnectionError: HTTPConnectionPool(host='172.17.141.197', port=9096): Max retries exceeded with url: //api/v1/query?query=sum(rate(container_cpu_usage_seconds_total%7Bcontainer_name!=%22POD%22,namespace=~%22openfaas.*%7Cdefault%22%7D%5B5m%5D))&time=1663935808.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff48a35cd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
#Pod Stats
expr_pod_cpu_aeneas_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_broker = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"emqx.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"tocloud.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_functions_all = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time)

print(expr_pod_cpu_aeneas_function)
metrics['pod_cpu_aeneas_function']=getdataprometheus(pre_url+expr_pod_cpu_aeneas_function)
metrics['pod_cpu_mosquitto_broker']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_broker)
metrics['pod_cpu_mosquitto_connector_aeneas']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_aeneas)
metrics['pod_cpu_mosquitto_connector_tocloud']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_tocloud)
metrics['pod_cpu_functions_all']=getdataprometheus(pre_url+expr_pod_cpu_functions_all)



In [ ]:
#!echo "workload_type,workload_size,Workload TPT,Average Workload TPT,Average FET,Average Waiting Time,C_CPU,C_RAM,C_disk_read,C_disk_write,C_network_recieve,C_network_transmit,pod_cpu_aeneas_function,pod_cpu_functions_all,pod_cpu_mosquitto_broker,pod_cpu_mosquitto_connector_aeneas,pod_cpu_mosquitto_connector_tocloud" > metrics.csv

In [ ]:
df_metrics = pd.read_csv("metrics.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)


In [ ]:
df_metrics.to_csv("metrics.csv",index=False)
#df_metrics=df_metrics.sort_values(by="workload_size")
display(df_metrics)

In [28]:
pod_metrics = {}
expr_pod_cpu_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['cpu_aeneas'] = getdataprometheus(pre_url+expr_pod_cpu_aeneas)
expr_pod_memory_aeneas = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['memory_aeneas'] = getdataprometheus(pre_url+expr_pod_memory_aeneas)
expr_pod_cpu_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['cpu_tocloud'] = getdataprometheus(pre_url+expr_pod_cpu_aeneas)
expr_pod_memory_tocloud = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-tolcoud.*"}['+str(interval)+'s]))&time='+str(time)
pod_metrics['memory_tocloud'] = getdataprometheus(pre_url+expr_pod_memory_aeneas)


ConnectionError: HTTPConnectionPool(host='172.17.141.197', port=9096): Max retries exceeded with url: //api/v1/query?query=sum(rate(container_cpu_usage_seconds_total%7Bcontainer_name!=%22POD%22,namespace=%22openfaas-fn%22,pod=~%22scaling-aeneas-mqtt.*%22%7D%5B60s%5D))&time=1663935808.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff48b0a970>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
#pod_metrics = json.loads(pod_metrics)
df2 = pd.DataFrame.from_dict(pod_metrics, orient="index")
print(df2)